<a href="https://colab.research.google.com/github/cyrus1123/AM/blob/main/plug_and_play1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pdb


In [1]:
import pandas as pd
import numpy as np

In [ ]:
# a = pd.DataFrame(np.random.random((4000, 121)))
# c = ['X_{}'.format(i) for i in range(120)]
# c.append('Y')
# a.columns = c
# a.to_excel('data.xlsx')

In [2]:
!pip install  transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.4 MB/s eta 0:00:00


#last working

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BigBirdTokenizer, BigBirdForSequenceClassification
from torch.optim import AdamW  # This line replaces the transformers import

# Specify the name of the pre-trained BigBird model you want to use
model_name = "google/bigbird-roberta-base"

# Load the tokenizer
tokenizer = BigBirdTokenizer.from_pretrained(model_name)

class LargeTextFileDataset(Dataset):
    def __init__(self, tokenizer, file_path, labels_path):
        self.tokenizer = tokenizer
        self.file_path = file_path
        self.labels_path = labels_path
        self.labels = self._read_labels(labels_path)
        self.texts = self._read_texts(file_path)

    def _read_labels(self, labels_path):
        with open(labels_path, 'r') as f:
            labels = [int(label.strip()) for label in f]
        return labels

    def _read_texts(self, file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            texts = [line.strip() for line in f]
        return texts

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = self.texts[idx]  # This should fetch the text from your dataset
        label = self.labels[idx]  # This should fetch the label from your dataset

        # Tokenize the text
        inputs = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        # Add the label to the inputs dictionary
        inputs['labels'] = torch.tensor(label, dtype=torch.long)

        return inputs

#****************************************************************************************
from sklearn.model_selection import train_test_split
import pandas as pd

class ExcelDataset(Dataset):
    def __init__(self, tokenizer, features, labels):
        self.tokenizer = tokenizer
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = ' '.join(map(str, self.features.iloc[idx, :]))
        inputs = self.tokenizer(text, padding='max_length', truncation=True, return_tensors="pt", max_length=512)
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        inputs["labels"] = torch.tensor(self.labels.iloc[idx], dtype=torch.long)
        return inputs

# Load the data from the Excel file
dataframe = pd.read_excel('data.xlsx')
features = dataframe.iloc[:, :-1]  # Assuming the last column is the label
labels = dataframe.iloc[:, -1]

# Split the data into training and testing sets
features_train, features_test, labels_train, labels_test = train_test_split(
    features, labels, test_size=0.2, random_state=42
)

# Create the datasets
train_dataset = ExcelDataset(tokenizer, features_train, labels_train)
test_dataset = ExcelDataset(tokenizer, features_test, labels_test)

# Define the DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

#*****************************************************************************************

# Define the collate_fn
def collate_fn(batch):
    batch_input_ids = [item['input_ids'] for item in batch]
    batch_attention_mask = [item['attention_mask'] for item in batch]
    batch_labels = [item['labels'] for item in batch]

    return {
        'input_ids': torch.stack(batch_input_ids),
        'attention_mask': torch.stack(batch_attention_mask),
        'labels': torch.stack(batch_labels)
    }

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)

# Initialize model
model = BigBirdForSequenceClassification.from_pretrained(model_name)

# Trainer class
class FCT_Trainer:
  def __init__(self, learning_rate, per_device_train_batch_size, per_device_eval_batch_size, n_epochs, weight_decay):
      self.learning_rate = learning_rate
      self.per_device_train_batch_size = per_device_train_batch_size
      self.per_device_eval_batch_size = per_device_eval_batch_size
      self.n_epochs = n_epochs
      self.weight_decay = weight_decay

  def train(self, model, train_loader, test_loader, epochs=None):
      optimizer = AdamW(model.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
      epochs = epochs or self.n_epochs
      gradient_accumulation_steps = 4  # Set this to the number of accumulation steps you want
      model.zero_grad()

      for epoch in range(epochs):
        model.train()
        total_loss = 0
        for step, batch in enumerate(train_loader):
          inputs = {key: value for key, value in batch.items() if key != "labels"}
          labels = batch["labels"]
          outputs = model(**inputs, labels=labels)
          loss = outputs.loss / gradient_accumulation_steps
          loss.backward()

          if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            model.zero_grad()

            total_loss += loss.item()

            avg_train_loss = total_loss / (len(train_loader) / gradient_accumulation_steps)
            print(f'Epoch {epoch+1}, Training Loss: {avg_train_loss:.10f}', total_loss)

        # Saving the model after every epoch
      torch.save(model.state_dict(), f'model_checkpoint_epoch_{epoch}.pt')

      # Evaluation (if needed)
      # ...
# Initialize trainer
trainer = FCT_Trainer(
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    n_epochs=3,
    weight_decay=0.01
)

# Start the training process
trainer.train(model, train_loader, test_loader)


Some weights of BigBirdForSequenceClassification were not initialized from the model checkpoint at google/bigbird-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Attention type 'block_sparse' is not possible if sequence_length: 512 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


Epoch 1, Training Loss: 0.0004377116 0.17508463561534882
Epoch 1, Training Loss: 0.0007718250 0.30873000621795654
Epoch 1, Training Loss: 0.0009330775 0.3732310011982918
Epoch 1, Training Loss: 0.0010328986 0.41315943375229836
Epoch 1, Training Loss: 0.0011735310 0.469412412494421
Epoch 1, Training Loss: 0.0012558409 0.5023363567888737
Epoch 1, Training Loss: 0.0012950819 0.5180327445268631
Epoch 1, Training Loss: 0.0013461444 0.5384577792137861
Epoch 1, Training Loss: 0.0014468051 0.5787220429629087


In [ ]:
probabilities = torch.softmax(outputs.logits, dim=1)
predictions = torch.argmax(probabilities, dim=1)


In [ ]:
import torch
from transformers import BigBirdTokenizer, BigBirdForSequenceClassification

# Create a small example to simulate the process
model_name = "google/bigbird-roberta-base"
tokenizer = BigBirdTokenizer.from_pretrained(model_name)
model = BigBirdForSequenceClassification.from_pretrained(model_name)

# Get the embedding layer's weights
embedding_weight = None
for name, param in model.named_parameters():
    if 'embeddings' in name and 'weight' in name:  # targeting the embedding layer's weights
        embedding_weight = param.data
        break

# Simulate a small batch of inputs
texts = ["example sentence for testing", "another example sentence"]
inputs = tokenizer(texts, padding='max_length', truncation=True, return_tensors="pt", max_length=128)

# Extract the input_ids from the inputs
input_ids = inputs['input_ids']

# Perform the embedding lookup
input_embeddings = torch.nn.functional.embedding(input_ids, embedding_weight)

# Check the shapes to confirm the operation was successful
input_ids_shape = input_ids.shape
input_embeddings_shape = input_embeddings.shape
print(input_ids_shape, input_embeddings_shape)


A_expanded = input_ids.unsqueeze(-1)

A_expanded * input_embeddings
